# Strain properties of Kramers chains

Here we want to see how a molecule unravells in the limit $Wi \gg 1$. Therefore simulations are done without Brownian force. 


In [1]:
import numpy as np
from dilutebrowniandynamics.simulate import simulate_batch
from dilutebrowniandynamics.molecules.Kramers_chain import KramersChain

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 8]

## Parameters definition

In [2]:
n_links = 1000       # Number of segments in the Kramers chain
n_ensemble = 4     # Number of molecules to simulate
n_proc = 4         # Number of processor cores to use

output_file = 'outputs/sandpit'

## Initialise molecules
Here we just draw random vectors from a normal distribution and rescale to unit vectors. To do this with call `from_normal_distribution` constructor. 

In [3]:
seq = np.random.SeedSequence(2022)
seeds = seq.spawn(n_ensemble)
# Starting vectors
molecules = [KramersChain.from_normal_distribution(n_links, seed)
             for seed in seeds]

## No thermalisation since Brownian forces are off

## Simulation
Now let's simulate a sudden uniaxial elongation.

In [ ]:
gradU = np.diag([.1, .05, .05])
dt = .01 
n_steps = 10000
write_interval = 10
print(f"Length of time series: {n_steps//write_interval}")
observables, molecules = simulate_batch(molecules, gradU, dt, n_steps, 
                                        write_interval=write_interval,
                                        no_average={'g_max', 'i_max', 'g_13', 'g_12', 'g_23'},
                                        n_proc=n_proc)

Length of time series: 1000
Physical time to compute: 100.0
Calculation started on 4 cores.


  0%|                                                     | 0/4 [00:00<?, ?it/s]

In [ ]:
t = np.arange(n_steps//write_interval)*dt*write_interval
A = observables['A_average']/n_links**2
trA = np.trace(A, axis1=1, axis2=2)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].semilogy(t, trA, label='tr(A)')
ax[0].axhline(1, ls='--', label='$Stratch$')
ax[0].axhline(1/n_links**0.5, ls=':', label='$Transistion$')
ax[0].axhline(1/n_links, label='$Coil$')

x = np.linspace(0,1)
ax[0].semilogy(x, np.exp(2*x - np.log(n_links)), label='Affine')

ax[0].legend()
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Value')
ax[0].set_title('Time series of the conformation tensor')


for g_max in observables['g_max'][0:1]:
    ax[1].semilogy(t, 8*g_max/n_links**2)
#ax[1].hist([REEs[:,0], REEs[:,1], REEs[:,2]], bins=bins, density=True, label=['x','y','z'])
#ax[1].legend()
#ax[1].set_title('Final end-to-end vectors distribution')
plt.show()

In [ ]:
gradUt = gradU(t)
plt.plot(gradUt[:,0,0], trA, '.', alpha=0.002)
plt.axhline(n_links**1.5, ls=':', label='$N^{1.5}$')

In [ ]:
0.52/0.5*0.0142

We can see that the average square length, tr(A), is constraint by (n_links)².

## Stress
We now examine the stress tensor.

In [ ]:
S = observables['S_average']
trS = np.trace(S, axis1=1, axis2=2)

fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].semilogy(t, trS, label='tr(S)')
ax[0].semilogy(t, S[:,0,0], label='Sxx)')
ax[0].set_title('Log Trace Stress')
ax[0].set_xlabel('Time')
ax[0].legend()
ax[1].plot(t, S[:,0,0], label='Sxx')
ax[1].plot(t, S[:,0,1], label='Sxy')
ax[1].plot(t, S[:,1,1], label='Syy')
ax[1].legend()
ax[1].set_xlabel('Time')
ax[1].set_title('Stress tensor')
plt.show()

A few remarks:
1. The stress is composed of a viscous part and visco-elastic part. When the flow is turned off, the viscous part vanishes instantaneously, while the viscoelastic part is relaxed over time.
2. The noise is very important with this estimator due to the rigid constaint.

In [ ]:
0.5/0.55*0.0148